<a href="https://colab.research.google.com/github/shivaprajapati34390-netizen/ML-project/blob/main/RAG_System_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers sentence-transformers faiss-cpu langchain langchain-community langchain-text-splitters

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
from google.colab import files

# This will open a file picker dialog to upload files
uploaded = files.upload()

Saving machine.txt to machine (1).txt


In [ ]:
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:

# 1. Initialize the Text Splitter
# This splitter is smart. It tries to split on paragraphs ("\n\n"),
# then newlines ("\n"), then spaces (" "), to keep semantically
# related text together as much as possible.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,  # Max size of a chunk
    chunk_overlap=20, # Overlap to maintain context between chunks
    length_function=len
)

In [ ]:
# 2. Create the chunks

# Get the content of the uploaded file and decode it from bytes to a string
file_name = 'machine (1).txt'
file_content = uploaded[file_name].decode('utf-8')

chunks = text_splitter.split_text(file_content)

print(f"We have {len(chunks)} chunks:")
for i, chunk in enumerate(chunks):
    print(f"""--- Chunk {i+1} ---
{chunk}
""")

We have 9 chunks:
--- Chunk 1 ---
Machine Learning is a branch of Artificial Intelligence that enables computers to learn from data and make decisions or predictions without being

--- Chunk 2 ---
without being explicitly programmed. Instead of following fixed instructions, machine learning systems use algorithms to identify patterns in data

--- Chunk 3 ---
patterns in data and improve their performance over time through experience. It mainly consists of three types: supervised learning, where models are

--- Chunk 4 ---
where models are trained using labeled data; unsupervised learning, where the system finds hidden patterns in unlabeled data; and reinforcement

--- Chunk 5 ---
and reinforcement learning, where an agent learns by receiving rewards or penalties based on its actions. Common algorithms include linear

--- Chunk 6 ---
include linear regression, decision trees, support vector machines, and neural networks. Machine learning is widely used in healthcare for disease

--- Chu

In [ ]:
# Step 3: Embeddings
# now we turn those text chunks into vectors we use a popular lightweight sentence transformer model.its brillient the understanding meaning of sentence

In [ ]:
from sentence_transformers import SentenceTransformer
# 1. Load the embedding model
# 'all-MiniLM-L6-v2' is a fantastic, fast, and small model.
# It runs 100% on your local machine.
model = SentenceTransformer('all-MiniLM-L6-v2')
#  2. Embed all our chunks
# This will take a moment as it "reads" and "understands" each chunk.
chunk_embeddings = model.encode(chunks)

print(f"Shape of our embeddings: {chunk_embeddings.shape}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Shape of our embeddings: (9, 384)


In [ ]:
#  vector store with FAISS
# we have our vector.now we need a database to store them in a way we can search by similarity.it is where faiss comes .its just few lines of code

In [ ]:
import faiss
import numpy as np



In [ ]:
# get the dimension of our vector
d = chunk_embeddings.shape[1]



In [ ]:
#  create the FAISS index
# indexFlatl2 is the simplest most basic index it calculates the exact distance(L2 distance )between our quary and vector.
index=faiss.IndexFlatL2(d)
#  add our chunk into embeddings to the index
# we must convert float32 into FAISS
index.add(np.array(chunk_embeddings).astype(np.float32))

In [ ]:
print(f"FAISS index created  with {index.ntotal} vectors")

FAISS index created  with 9 vectors


In [ ]:
# step 5 retrieve,Augment,generate (RAG)
# this is a final part here will user ask aquestion.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the tokenizer and model for flan-t5-small
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-small')
model_generator = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-small')

config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/190 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# this is our RAG pieline function

In [ ]:
def answer_question(quary):
  # RETRIEVE
  # embedd the user quary
  quary_embedding=model.encode([quary]).astype('float32')
  # Search the faiss index into top (e.g,k=2) most similar chunk
  k=2
  distance,indices=index.search(quary_embedding,k)
  # get the actual text chunk from our original chunk list
  retrieved_chunks=[chunks[i] for i in indices[0]]
  context="\n\n.".join(retrieved_chunks)
  # 2 Augment
  # this is the magic propmt we combined the retrieved chunks with the uers quary
  prompt_template= f"""
  answer the following question using only the provided context.
  if the answer is not in the context,say \"i don't have any information\"

  Context:
  {context}

  Question:
  {quary}
  Answer:
  """
  # 3 Generate
  # Encode the prompt
  input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids

  # Generate output
  outputs = model_generator.generate(input_ids, max_new_tokens=100, do_sample=False)
  answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

  print(f"---CONTEXT----\n{context}\n")
  return answer

In [ ]:
# Example usage of the answer_question function
question_to_ask = "What is machine learning?"
response = answer_question(question_to_ask)
print(f"\n---GENERATED ANSWER---\n{response}")

---CONTEXT----
Machine Learning is a branch of Artificial Intelligence that enables computers to learn from data and make decisions or predictions without being

.without being explicitly programmed. Instead of following fixed instructions, machine learning systems use algorithms to identify patterns in data


---GENERATED ANSWER---
a branch of Artificial Intelligence that enables computers to learn from data and make decisions or predictions without being .without being explicitly programmed


In [ ]:
# Example usage of the answer_question function
question_to_ask = "What is machine learning?"
response = answer_question(question_to_ask)
print(f"\n---GENERATED ANSWER---\n{response}")

---CONTEXT----
Machine Learning is a branch of Artificial Intelligence that enables computers to learn from data and make decisions or predictions without being

.without being explicitly programmed. Instead of following fixed instructions, machine learning systems use algorithms to identify patterns in data


---GENERATED ANSWER---
a branch of Artificial Intelligence that enables computers to learn from data and make decisions or predictions without being .without being explicitly programmed


In [ ]:
print("Hello! I am a chatbot based on your document. Ask me anything about it!")
print("Type 'exit' or 'quit' to end the conversation.")

while True:
    user_question = input("\nYour Question: ")
    if user_question.lower() in ['exit', 'quit']:
        print("Goodbye!")
        break

    response = answer_question(user_question)
    print(f"\nChatbot: {response}")

Hello! I am a chatbot based on your document. Ask me anything about it!
Type 'exit' or 'quit' to end the conversation.

Your Question: what is machine learning
---CONTEXT----
Machine Learning is a branch of Artificial Intelligence that enables computers to learn from data and make decisions or predictions without being

.Although it requires large amounts of data and computational power, machine learning plays a vital role in modern technology and continues to


Chatbot: a branch of Artificial Intelligence that enables computers to learn from data and make decisions or predictions without being
